# Installing needed libraries for visualizations by pip

In [34]:
!pip uninstall matplotlib -y
!pip install matplotlib==3.1.3
!pip install geopandas
!pip install folium
!pip install altair
!pip3 install ipympl
!pip3 install matplotlib

Found existing installation: matplotlib 3.7.0
Uninstalling matplotlib-3.7.0:
  Successfully uninstalled matplotlib-3.7.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached matplotlib-3.1.3-cp38-cp38-manylinux1_x86_64.whl (13.1 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipympl 0.9.3 requires matplotlib<4,>=3.4.0, but you have matplotlib 3.1.3 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached matplotlib-3.7.0-cp38-c

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [35]:
import pandas as pd
import geopandas
import matplotlib.pyplot as plt
import requests
import numpy as np
import folium
import matplotlib.pyplot as plt
import plotly.express as px

# **Extrating Data Through API**



in the cell below there is a class that you can pass in the main url and the api key once you made the object you can use the method and give it the extension url and the params then you will receive data

In [36]:

class main:
  
  def __init__(self, api, endpoint):
    self.headers = {"Authorization": "Bearer " + api}
    self.endpoint = endpoint

class apis(main):

  def get(self,url,**params):
    """
    **params** 
    year 
    classification  (HighSchool, JUCO, PrepSchool)
    position Position abbreviation filter
    state State or province abbreviation filter
    team Committed team filter (required if year not specified)
    """

    #add at least (year,team) in the params)
    
    return requests.get(self.endpoint + url ,params = params, headers=self.headers).json()


**Requesting the recruiting players dataset through the api object**
setting the param key year to a range from 2011 to 2022 included
reading the data by json_normalize it makes one of the columns not having a json objects it will split them up to more than one column in the dataframe and I did Concat each dataframe to get all the years required

In [38]:
# making an api object
ob = apis("api","https://api.collegefootballdata.com/")

# getting recruited players for the year 2022
# df = pd.DataFrame(ob.get("recruiting/players",year=2022))
df = pd.concat([pd.json_normalize(ob.get("recruiting/players",year=year)) for year in range(2011, 2023+ 1)])


# Saving the primary dataset locally
and showing a sample of it

In [39]:
df.head()

,id,athleteId,recruitType,year,ranking,name,school,committedTo,position,height,weight,stars,rating,city,stateProvince,country,hometownInfo.latitude,hometownInfo.longitude,hometownInfo.fipsCode
0,19571,-1032080,HighSchool,2011,1.0,Jadeveon Clowney,South Pointe,South Carolina,WDE,77.0,255.0,5,1.0000,Rock Hill,SC,USA,34.9248667,-81.0250784,45091
1,19572,-1000934,HighSchool,2011,2.0,Cyrus Kouandjio,DeMatha Catholic,Alabama,OT,79.0,315.0,5,0.9993,Hyattsville,MD,USA,38.9529442,-76.9408647,24033
2,19573,-1016566,HighSchool,2011,3.0,La'el Collins,Redemptorist,LSU,OT,77.0,306.0,5,0.9976,Baton Rouge,LA,USA,30.4459596,-91.18738,22033
3,19574,517390,HighSchool,2011,4.0,Karlos Williams,Ridge Community,Florida State,S,74.0,210.0,5,0.9976,Davenport,FL,USA,28.1614046,-81.6017417,12105
4,19575,517222,HighSchool,2011,5.0,Anthony Johnson,O Perry Walker,LSU,DT,75.0,294.0,5,0.9973,New Orleans,LA,USA,29.9499323,-90.0701156,22071


In [40]:
df.dropna(subset=['committedTo','school','name','country','hometownInfo.latitude','hometownInfo.longitude'],inplace=True)
df.drop_duplicates( keep='last',inplace=True)

# Downloading the secondary dataset 
one dataset can be downloaded from the cell below the other one needs to be downloaded through the link and move it to the project directory 

In [41]:
#downloading the files income data based on city and cities based on coordinates

"""
you will have to download the cities based on coordinates data
here is the link to it and move the file to the project directory

https://www.unitedstateszipcodes.org/zip_code_database.csv?download_auth=a5443d423dfb4db4fe077643f04b2f8c

"""
!wget https://www.irs.gov/pub/irs-soi/19zpallagi.csv

--2023-02-21 17:52:56--  https://www.irs.gov/pub/irs-soi/19zpallagi.csv
Resolving www.irs.gov (www.irs.gov)... 104.99.89.171, 2600:1409:9800:78b::f50, 2600:1409:9800:794::f50
Connecting to www.irs.gov (www.irs.gov)|104.99.89.171|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘19zpallagi.csv.1’

19zpallagi.csv.1        [             <=>    ] 190.40M  22.6MB/s    in 10s     

2023-02-21 17:53:07 (19.1 MB/s) - ‘19zpallagi.csv.1’ saved [199650533]



The 19zpallagi dataset has only the income based on zip codes and for the zip_code_database dataset has zip codes, cities, coordinates, etc.


In [43]:
# IRS income dataset
income = pd.read_csv("data/19zpallagi.csv")

# Coordinates with zip codes
zips_coordinates = pd.read_csv("data/zip_code_database.csv")

In [44]:
print(income.columns)

Index(['STATEFIPS', 'STATE', 'zipcode', 'agi_stub', 'N1', 'mars1', 'MARS2',
       'MARS4', 'ELF', 'CPREP',
       ...
       'N85300', 'A85300', 'N11901', 'A11901', 'N11900', 'A11900', 'N11902',
       'A11902', 'N12000', 'A12000'],
      dtype='object', length=152)


In [45]:
# isolating and removing non-US recruits
non_us_players = [df['hometownInfo.latitude'].isna()]
df = df[df['hometownInfo.latitude'].isna() == False]

In [46]:
# slimming down dfs for visual ease
relevant_columns_income = ['STATE', 'zipcode', 'A02650']

relevant_columns_linking_info = ['state', 'zip', 'primary_city', 'latitude', 'longitude']

income = income[relevant_columns_income]

zips_coordinates = zips_coordinates[relevant_columns_linking_info]

Here I did merge two datasets the income one and zip_coordinates by zipcode then did groupby by zipcode and applied mean to the average income per household to get the mean income per household in a city 

In [47]:
# avg income by zipcode
median_income_by_zipcode = income.merge(zips_coordinates, left_on=['zipcode'], right_on=['zip']).groupby("zipcode").A02650.mean().reset_index()


In [48]:
# merging with other info
merged_income_zipcor = zips_coordinates.merge(median_income_by_zipcode, left_on=['zip'], right_on=['zipcode'])

In [49]:
# adjusting coordinates for merging
merged_income_zipcor['adj_lat'] = merged_income_zipcor['latitude'].apply(lambda x: float(str(x).split(".")[0]))
merged_income_zipcor['adj_lng'] = merged_income_zipcor['longitude'].apply(lambda x: float(str(x).split(".")[0]))

In [50]:
# getting average income by city
income_by_city_state_coord = merged_income_zipcor.groupby(['primary_city', 'state', 'adj_lat', 'adj_lng']).A02650.mean().reset_index()
income_by_city_state_coord['A02650'] = income_by_city_state_coord['A02650'].round()
income_by_city_state_coord = income_by_city_state_coord.rename(columns={"A02650" : "Average Income"})


Here I Changed the type of the two columns latitude and longitude. 
to float and did rounded it down 

In [51]:
df['lat_adj'] = df['hometownInfo.latitude'].apply(lambda x: float(str(x).split(".")[0]))
df['lng_adj'] = df['hometownInfo.longitude'].apply(lambda x: float(str(x).split(".")[0]))


In [52]:
df.head()

,id,athleteId,recruitType,year,ranking,name,school,committedTo,position,height,...,stars,rating,city,stateProvince,country,hometownInfo.latitude,hometownInfo.longitude,hometownInfo.fipsCode,lat_adj,lng_adj
0,19571,-1032080,HighSchool,2011,1.0,Jadeveon Clowney,South Pointe,South Carolina,WDE,77.0,...,5,1.0000,Rock Hill,SC,USA,34.9248667,-81.0250784,45091,34.0,-81.0
1,19572,-1000934,HighSchool,2011,2.0,Cyrus Kouandjio,DeMatha Catholic,Alabama,OT,79.0,...,5,0.9993,Hyattsville,MD,USA,38.9529442,-76.9408647,24033,38.0,-76.0
2,19573,-1016566,HighSchool,2011,3.0,La'el Collins,Redemptorist,LSU,OT,77.0,...,5,0.9976,Baton Rouge,LA,USA,30.4459596,-91.18738,22033,30.0,-91.0
3,19574,517390,HighSchool,2011,4.0,Karlos Williams,Ridge Community,Florida State,S,74.0,...,5,0.9976,Davenport,FL,USA,28.1614046,-81.6017417,12105,28.0,-81.0
4,19575,517222,HighSchool,2011,5.0,Anthony Johnson,O Perry Walker,LSU,DT,75.0,...,5,0.9973,New Orleans,LA,USA,29.9499323,-90.0701156,22071,29.0,-90.0


In [53]:
# joining together for final df
final = df.merge(income_by_city_state_coord, left_on=['city', 'stateProvince', 'lat_adj', 'lng_adj'],
                                       right_on=['primary_city', 'state', 'adj_lat', 'adj_lng'])

# Adding a column a color column and geometry column

the first column color giving green color for the high income level, blue for the middle income level and red for the low income level

the second column will be geometry column it will be for ploting with follium it has the latitude nad longitude. 

In [54]:
top_10 = final['Average Income'].quantile(0.9)
print("Top 10%", top_10)
top_25 = final['Average Income'].quantile(0.75)
print("Top 25%", top_25)
top_50 = final['Average Income'].quantile(0.5)
print("Top 50%", top_50)
top_75 = final['Average Income'].quantile(0.25)
print("Top 75%", final['Average Income'].quantile(0.25))
top_90 = final['Average Income'].quantile(0.1)

Top 10% 391258.0
Top 25% 257687.0
Top 50% 169904.0
Top 75% 102868.0


In [55]:
col = 'Average Income'
conditions = [final[col]>=top_10,
              (final[col]<top_10)&(final[col]>=top_25),
              (final[col]<top_25)&(final[col]>=top_50),
             (final[col]<top_50)&(final[col]>=top_75),
            final[col]<top_75]

choices = ['top 10 pct', "top quartile", "second quartile", 'third quartile','fourth quartile']
final['Quartile Level'] = np.select(conditions,choices, default=np.nan)

In [56]:
gdf = geopandas.GeoDataFrame(final, geometry=geopandas.points_from_xy(final['hometownInfo.longitude'], final['hometownInfo.latitude']))
gdf.head()

,id,athleteId,recruitType,year,ranking,name,school,committedTo,position,height,...,hometownInfo.fipsCode,lat_adj,lng_adj,primary_city,state,adj_lat,adj_lng,Average Income,Quartile Level,geometry
0,19571,-1032080,HighSchool,2011,1.0,Jadeveon Clowney,South Pointe,South Carolina,WDE,77.0,...,45091,34.0,-81.0,Rock Hill,SC,34.0,-81.0,269076.0,top quartile,POINT (-81.02508 34.92487)
1,20160,512424,HighSchool,2011,591.0,Justin Worley,Northwestern,Tennessee,PRO,76.0,...,45091,34.0,-81.0,Rock Hill,SC,34.0,-81.0,269076.0,top quartile,POINT (-81.02508 34.92487)
2,20244,-1007350,HighSchool,2011,676.0,Roderick Byers,Northwestern,Clemson,SDE,75.0,...,45091,34.0,-81.0,Rock Hill,SC,34.0,-81.0,269076.0,top quartile,POINT (-81.02508 34.92487)
3,20875,-1032025,HighSchool,2011,1345.0,Gerald Dixon Jr.,Northwestern,South Carolina,DT,74.0,...,45091,34.0,-81.0,Rock Hill,SC,34.0,-81.0,269076.0,top quartile,POINT (-81.02508 34.92487)
4,21202,-1032025,HighSchool,2011,1672.0,Gerald Dixon,South Pointe,South Carolina,SDE,74.0,...,45091,34.0,-81.0,Rock Hill,SC,34.0,-81.0,269076.0,top quartile,POINT (-81.02508 34.92487)


In [57]:

count_committedTo_income = pd.DataFrame({'count' : final.groupby( ["Quartile Level"] ).size()}).reset_index()
count_committedTo_income['percent_quintile'] = count_committedTo_income['count']/count_committedTo_income['count'].sum()

In [62]:

def geo_plot(gdf):
  # px.set_mapbox_access_token("MAP_API")
  fig = px.scatter_mapbox(gdf,
                          lat=gdf.geometry.y,
                          lon=gdf.geometry.x,
                          hover_name="name",
                          hover_data=["rating", "year", "school"],
                          color="Quartile Level",
                          zoom=1)

  fig.update_layout(legend = dict(font=dict(size=20), bordercolor='rgb(100,100,100)',
                                  borderwidth=2,
                                  itemclick='toggleothers',# when you are clicking an item in legend all that are not in the same group are hidden
                                  x=0.750,
                                  y=1))

  fig.update_layout(mapbox_style="open-street-map")

  fig.show()

geo_plot(gdf[gdf['stars'] >= 4 ])



# Plotting all players on the map and colors represents the income level

In [63]:
geo_plot(gdf)

Output hidden; open in https://colab.research.google.com to view.

# plotting a map plot filterd by a specific team 

In [60]:
#plotting here for UCLA team that has players with an average of 3.5 stars and above

geo_plot(gdf[gdf['committedTo'] == "UCLA"])


In [61]:
#plotting for Georgia team without filter on stars only filtered by the difference in income levels

geo_plot(gdf[gdf['committedTo'] == "Georgia"])

